# Malware Simulation Demo / Happy Holidays

. | .
- | - 
![alt](gift.png) | ![alt](Element-AI-Symbol-RGB.jpg)

This notebook shows how to run a simple ITSim simulation and collect its telemetry data from a datastore server. Then it quickly overlooks the generated data by showing the telemetry records and plotting the chronological graphs of events.


## Step 1:  Launch a Datastore Server to collect simulation data

This server collects ITSim telemetry and logs over a REST API and archives them into a Database (SQLite db). This server is running throughout the simulations. It could be used to collect data from simulations running simultaneously on multiple machines.    

In [ ]:
import os

DB_FILE = "malware_simulation_01.sqlite"
HOSTNAME = "localhost"
SERVER_PORT = "5000"

if os.system(f'python ../../bin/itsim_serve_datastore.py --sqlite_file {DB_FILE} --host {HOSTNAME} --port {SERVER_PORT} &') == 0:
    print(f'Datastore server is running:  http://{HOSTNAME}:{SERVER_PORT}')

## Step 2: Set up the simulation world

Define simulation parameters. We consider here a simple flat local network linked to the Internet. One of the endpoints on the local network will get a backdoor to run, which beacons home at intervals distributed according to a uniform distribution. The number of simulations to run, the duration of the simulation (in sim time) and the number of endpoints deployed in the local network are all parameterizable.

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual
def set_simulation_parameters(sim_runs, sim_lenght, network_topology,nb_endpoints, backdoor_random_start):
    return {'sim_runs':sim_runs,
            'sim_len':sim_lenght,
            'network_topology':network_topology,
            'nb_endpoints':nb_endpoints,
            'backdoor_interval_bounds':backdoor_random_start}
            
style={'description_width': 'initial'}
sim_runs_widget = widgets.IntSlider(value=4,
                                    min=1,
                                    max=10,
                                    description='Simulations:',
                                    style=style)
sim_lenght_widget = widgets.IntSlider(value=120,
                                    min=5,
                                    max=300,
                                    description='Sim. lenght (min):',
                                    style=style)
network_topology_widget = widgets.Dropdown(options=['Flat + Internet'],
                                           value='Flat + Internet',
                                           description='Topology:',
                                           style=style)
nb_endpoints_widget = widgets.IntSlider(value=20,
                                        min=2,
                                        max=65000,
                                        description='Endpoints:',
                                        style=style)
backdoor_random_start_widget = widgets.FloatRangeSlider(value=[15.0, 45.0],
                                                   min=1.0,
                                                   max=100.0,
                                                   description='Beacon interval U(a,b):',
                                                   style=style)
sim = interactive(set_simulation_parameters, 
                sim_runs=sim_runs_widget, 
                sim_lenght=sim_lenght_widget, 
                network_topology=network_topology_widget,
                nb_endpoints=nb_endpoints_widget,
                backdoor_random_start = backdoor_random_start_widget)
display(sim)

Define a simulation runner. If found useful, this routine could be pulled back into the ITsim library.

In [ ]:
def run_simulations(sim_cfg, sim_fct):
    print(f"Running {sim_cfg.result['sim_runs']} simulations based on the following configuration:")
    print(f"\t- Simulation Lenght: \t\t\t\t\t{sim_cfg.result['sim_len']}")
    print(f"\t- Network Topology: \t\t\t\t\t{sim_cfg.result['network_topology']}")
    print(f"\t- Number of servers: \t\t\t\t\t1")
    print(f"\t- Number of Endpoints: \t\t\t\t\t{sim_cfg.result['nb_endpoints']}")
    print(f"\t- Random backdoor callback based on distribution: \tUniform({sim_cfg.result['backdoor_interval_bounds'][0]},{sim_cfg.result['backdoor_interval_bounds'][1]})")
    bar = widgets.IntProgress(value=0,
                              min=0,
                              max=sim_cfg.result['sim_runs'],
                              step=1,
                              description='Progress:',
                              bar_style='info', 
                              orientation='horizontal')
    display(bar)
    for _ in range(sim_cfg.result['sim_runs']):
        sim_fct(sim_cfg.result)
        bar.value+=1

A few declarations, plus global parameter definitions. We have on our to-do list to pull all the most important classes and functions in a single root module.

In [ ]:
import random
from greensim.random import constant, normal, uniform, bounded
from itsim import malware
from itsim.datastore.datastore import DatastoreClientFactory
from itsim.software.context import Context
from itsim.machine.endpoint import Endpoint
from itsim.machine.socket import Timeout
from itsim.network.location import Location
from itsim.network.link import Link
from itsim.network.router import Router
from itsim.network.route import Relay
from itsim.simulator import Simulator, now, advance
from itsim.types import as_address, as_cidr, Protocol
from itsim.units import B, S, MS, MbPS, MIN

PORT_C2 = 443
size_beacon = bounded(normal(128 * B, 32 * B), lower=32 * B)
address_c2 = as_address("120.11.12.20")
local_cidr = as_cidr("10.11.0.0/16")
internet_cidr = as_cidr("0.0.0.0/0")

The backdoor process, which will run on a randomly selected endpoint. Note: the advance commands have been added to help visualizing data; they're not required to properly implement the simulation. 


In [ ]:
@malware
def backdoor(context: Context, interval_lower: float, interval_upper: float) -> None:
    interval_beacons = uniform(interval_lower * MIN, interval_upper * MIN)
    while True:
        with context.node.bind(Protocol.TCP) as socket:
            advance(200 * MS)
            socket.send(Location(address_c2, PORT_C2), next(size_beacon), {"content": "ping"})
            try:
                packet = socket.recv(10 * S)
                advance(200 * MS)
            except Timeout:
                print("TIMEOUT -- We might have a bug, take a look.")  # Try again next interval...
        advance(next(interval_beacons))



The command and control server software. This will be run on an endpoint set up outside of the local network.


In [ ]:
@malware
def command_and_control(context: Context) -> None:
    with context.node.bind(Protocol.UDP, PORT_C2) as socket:
        while True:
            packet = socket.recv()
            advance(200 * MS)
            socket.send(packet.source, 8, {"content": "pong"})

The next function instantiates the simulated *world* and runs the simulation inside it.


In [ ]:
def run_backdoor_simulation(sim_cfg):
    sim = Simulator()
    DatastoreClientFactory().sim_uuid = sim.uuid

    local = Link(local_cidr, latency=uniform(1 * MS, 5 * MS), bandwidth=constant(100 * MbPS))
    internet = Link(internet_cidr, latency=constant(200 * MS), bandwidth=constant(1000 * MbPS))

    router = Router()
    router.connected_to_static(local, 1)
    router.connected_to_static(internet, "1.2.3.4")

    route_local_to_internet = Relay(router._interfaces[local_cidr].address, internet_cidr)
    route_internet_to_local = Relay(router._interfaces[internet_cidr].address, local_cidr)

    endpoints = [Endpoint().connected_to_static(local, n + 10, [route_local_to_internet]) for n in range(sim_cfg["nb_endpoints"])]
    endpoints[random.randint(0, sim_cfg["nb_endpoints"] - 1)].run_proc_in(
        sim,
        0.1,
        backdoor,
        sim_cfg["backdoor_interval_bounds"][0],
        sim_cfg["backdoor_interval_bounds"][1]
    )

    host_c2 = Endpoint().connected_to_static(internet, address_c2, [route_internet_to_local])
    host_c2.run_proc(sim, command_and_control)

    sim.run(sim_cfg["sim_len"] * MIN)

## Step 3: Run the simulation


In [ ]:
run_simulations(sim, run_backdoor_simulation)

We're done with the datastore server, we can close it. 

In [ ]:
import requests

try: 
    response = requests.post(f'http://{HOSTNAME}:{SERVER_PORT}/stop')
    print("Server properly shutdown")
except:
    print("Can't reach the server to shut it down")

## Step 4: Retrieve Simulation Telemetry

Now that the simulations ran to completion, we can access the data collected by the datastore (SQLite database).

The datastore is storing telemetry events as JSON strings into a SQLite database (for the convenience of quick prototyping). This is a quick preview of what the data looks like using Pandas. 

In [ ]:
import sqlite3
import pandas as pd
import json
from texttable import Texttable
conn = sqlite3.connect(DB_FILE)
df = pd.read_sql_query("SELECT * FROM network_event;", conn)
df

From this dataframe, we can load the json data and list the fields of the telemetry events: 

In [ ]:
simulations = df.sim_uuid.unique()
print(f"Telemetry events for {len(simulations)} simulations")
df_json = []

simulations_data = {}
for sim in simulations:
    df2 = df.loc[df['sim_uuid'] == sim]
    df2 = df2.set_index("timestamp", drop = False)
    df1 = df2.loc[:, 'json'].to_frame()
    entry_list = []
    for _, row in df1.iterrows():
        entry_list.append(json.loads(row.json))
    simulations_data[sim] = entry_list

for sim_uuid, telemetry_list in simulations_data.items():
    print(f"\nSimulation {sim_uuid}\n")
    t = Texttable()
    t_rows = []
    t_header = ['Timestamp', 'Telemetry uuid', 'Node uuid', 'Event', 'Tags', 'Prot.', 'Pid', 'Src', 'Dst']
    t_rows.append(t_header)   
    for telemetry in telemetry_list:
        t_rows.append([telemetry["timestamp"],
                       telemetry["uuid"],
                       telemetry["uuid_node"],
                       telemetry["network_event_type"],
                       telemetry["tags"],
                       telemetry["protocol"],
                       telemetry["pid"],
                       telemetry["src"],
                       telemetry["dst"]])
    t.set_cols_width([13, 18, 13,5,11,5, 4, 10, 10])
    t.set_cols_dtype(["t","t","t","t","t","t","t","t","t"])
    t.add_rows(t_rows)
    print(t.draw())
              

## Step 5: Plot Telemetry Data
Here is a "quick and dirty approach" to plot telemetry events chronologically: 

In [ ]:
import ast
import time
import random
import datetime
import dateutil.parser
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.figure_factory as ff
print(__version__) # requires version >= 1.9.0

def plot_simulation_telemetry_events(conn, sim_uuid):
    node_uuid_idx = 0
    node_timestamp_idx = 1
    node_simulation_idx = 2
    node_json_idx = 3
    df = []

    print(f"Simulation: {sim_uuid}")
    with conn:
        cursor = conn.cursor()
        cursor.execute(f'SELECT * FROM network_event WHERE "sim_uuid"="{sim_uuid}"')
        all_entries = cursor.fetchall()    
        cnt = 0
        colors = []
        net_events_post_proc = []

        for network_event in all_entries:
            data = ast.literal_eval(network_event[node_json_idx])      
            uuid_node = data["uuid_node"]
            timestamp = data["timestamp"]
            datetime_start = dateutil.parser.parse(timestamp)
            datetime_stop = datetime_start + datetime.timedelta(milliseconds=100)
            timestamp_stop = datetime_stop.isoformat()
            uuid_telemetry = data["uuid"]
            
            colors = {'open': (0.59, 0.25, 0.72),
                      'close': (0.37, 0.72, 0.25),
                      'send': (0.25, 0.72, 0.60),
                      'recv': (0.72, 0.25, 0.37)}
            
            df.append(dict(Task=uuid_node, Start=timestamp, Finish=timestamp_stop, Type=data["network_event_type"], Event_Name="Net. Events"))

        init_notebook_mode(connected=True)
        fig = ff.create_gantt(df, title="ITsim Network Events", colors=colors, index_col='Type', showgrid_x=True, showgrid_y=True, show_colorbar=True, group_tasks=True)
        iplot(fig)

We're now calling this function to plot an event chart for each simulation. 

*WARNING*: the ratio between the simulation run time and each event's length is very high, zooming is thus required to visualize the events (Plotly controls at the top right of each chart allow to zoom in and out). 


This snapshot shows what you can expect from zooming in a data transfer region:
[snapshot](snapshot.png)


In [ ]:
for sim_uuid in simulations:
    plot_simulation_telemetry_events(conn, sim_uuid)

# THANKS!!
